In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver   
from requests import get
import time
import re
import csv

In [ ]:
url = "https://www.blockchain.com/btc/blocks"
driver = webdriver.Chrome('/Users/wufangzheng/Downloads/chromedriver') #set the webdriver to Chrome driver
driver.get(url)                                                        #now we can use selenium to access this website
time.sleep(10)                                                         #wait to let the selenium load the website

In [ ]:
#find the most recent block
element = driver.find_element_by_tag_name("tbody")     #find the first tag named 'tbody'

content = element.find_element_by_tag_name('a') #in the class, find the first link which is the link of the block
link = content.get_attribute('href')            #get the link            
blockNumber = int(re.findall('\d{6}',link)[0])  #get the link number by regular expression
print(blockNumber)

In [ ]:
#loop for 100 blocks
for i in range(100):
    print('Now scraping block No.' + str(blockNumber) + ': ')
    
    #get in the website of each block
    while True:
        html = get('https://www.blockchain.com/btc/block-height/' + str(blockNumber))
        if html.status_code == 200:
            print('Block Accessed!')
            break
        else:
            print('Access Denied, Reconnect in 5 seconds...')
            time.sleep(5)
    
    soup_object = BeautifulSoup(html.text,'html.parser')
    
    newstr = soup_object.findAll('td')     #all information we want is under the tag 'td'
    
    #on this website, we can find hash of this block
    #by clicking that hash on the website, we can access all transactions in this block
    blockHash = re.findall('/btc/block/\w*', str(newstr))[0]    #get the block hash
    
    #go to the website that contains all transaction info
    url = 'https://www.blockchain.com' + blockHash         
    driver.get(url)
    time.sleep(10)                         #wait to load
    
    #get the infomation of mining pool
    pool = driver.find_elements_by_tag_name('td')
    print('Relayed By: ' + pool[15].text + '\n')
    
    #find urls of each transaction
    transactionSet = driver.find_elements_by_css_selector('a.hash-link')
    
    #clean and add those urls to a set
    links = []
    for element in transactionSet:
        link = element.get_attribute('href')
        if len(link)<=100:             #all urls of transaction have exactly 98 characters
            links.append(link)
    print('There are ' + len(links) + ' transactions in this block')
    
    #look into each transaction and fetch info
    for link in links:
        while True:
            html = get(link)
            if html.status_code == 200:
                break
            else:
                print('Access Denied, Reconnect in 5 seconds...')
                time.sleep(5)
        soup_object = BeautifulSoup(html.text,'html.parser')
        
        #get the hash of this transaction
        tHash = re.findall('\w{64}', link)
        print('Transaction Hash: ' + tHash[0] + '\n')
        
        #get the time of this transaction
        rtime = re.findall('\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d', soup_object.prettify())
        print('Received Time: ' + rtime[0] + '\n')
        
        #get the value of this transaction
        value = soup_object.find_all("button", "btn btn-success cb")
        print ('Value: ' + re.findall('\d+[.]?[\d]* BTC', str(value))[0] + '\n')
        
        #get the senders' info of this transaction
        #there may be multiple senders
        sender = soup_object.find_all("td", "txtd hidden-phone mobile-f12 stack-mobile")
        sender = re.findall('/btc/address/\w*', str(sender))
        i = 1
        
        #if there is no sender
        if len(sender) == 0:
            print('No Inputs (Newly Generated Coins)')
            
        #if there is/are senders:
        for element in sender:
            print('Sender ' + str(i) + ': ' + re.sub('/btc/address/', '', element))
            i+=1
        print()
        
        #get the receivers' info of this transaction
        #there may be multiple receivers
        receiver = soup_object.find_all("td", "txtd mobile-f12 stack-mobile")
        receiver = re.findall('/btc/address/\w*', str(receiver))
        i = 1
        
        #if there is no receiver:
        if len(receiver) == 0:
            print('Unable to decode output address')
            
        #if there is receiver:
        for element in receiver:
            print('Receiver ' + str(i) + ': ' + re.sub('/btc/address/', '', element))
            i+=1
        print('\n\n')

        
    blockNumber -= 1